In [39]:
import tensorflow as tf
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical #Similar to one_hot_encoding, but converts into matrix of 1 and 0s instead of a column of 1s and 0s.
import tensorflow_datasets as tfds

In [40]:
(train, test) = tfds.load('mnist',
                                split = ['train','test'])

In [41]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

def load_dataset(train_size, test_size):
    train, test = tf.keras.datasets.mnist.load_data()

    # take a sample
    train_idx = np.random.randint(low=0, high=train[0].shape[0], size=train_size)
    test_idx = np.random.randint(low=0, high=test[0].shape[0], size=test_size)
    X_train = train[0][train_idx].reshape(-1,28*28)
    y_train = train[1][train_idx].reshape(-1,1)
    X_test = test[0][test_idx].reshape(-1,28*28)
    y_test = test[1][test_idx].reshape(-1,1)

    scale = StandardScaler()
    X_train = scale.fit_transform(X_train)
    X_test = scale.transform(X_test)

    OH = OneHotEncoder(categories='auto', sparse=False)
    y_train = OH.fit_transform(y_train)
    y_test = OH.transform(y_test)

    print('X_train:',X_train.shape)
    print('y_train:',y_train.shape)
    print('X_test:',X_test.shape)
    print('y_test:',y_test.shape)
    print('Min:', X_train.min())
    print('Max:', X_train.max())

    return X_train, X_test, y_train, y_test

In [42]:
X_train, X_test, y_train, y_test = load_dataset(60000,10000)

X_train: (60000, 784)
y_train: (60000, 10)
X_test: (10000, 784)
y_test: (10000, 10)
Min: -1.2723338739191787
Max: 244.94693302850916


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [43]:
X_train.shape

(60000, 784)

In [44]:
X_test.shape #Have to change the shape of the train/ test

(10000, 784)

In [45]:
X_train = X_train.reshape(X_train.shape[0],28,28,1)
X_test = X_test.reshape(X_test.shape[0],28,28,1)

In [46]:
from tensorflow.keras import layers

In [47]:
def simple_model(initializer, activation='relu'):
    return tf.keras.Sequential([layers.Dense(32, activation, input_shape=(28,28,1), kernel_initializer=initializer),
                                layers.Dense(32, activation, kernel_initializer=initializer),
                                layers.Dense(32, activation, kernel_initializer=initializer),
                                layers.Dense(32, activation, kernel_initializer=initializer),
                                layers.Dense(10, activation = 'softmax', kernel_initializer=tf.keras.initializers.glorot_normal())
    ])

In [48]:
alexnet_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(48,(3,3), activation='relu', padding='same', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(192, (3,3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(192, (3,3), activation='relu', padding='same'),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5), #after every Dense layer we add. adropout layer to randomly drop =
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10, activation='softmax') #10 for categories, end with softmax
])

In [49]:
alexnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [50]:
alexnet_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 28, 28, 48)        480       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 14, 14, 48)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 14, 14, 128)       55424     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 7, 7, 128)         0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 7, 7, 192)         221376    
                                                                 
 conv2d_8 (Conv2D)           (None, 7, 7, 192)        

In [51]:
alexnet_history = alexnet_model.fit(
    X_train, y_train,
    epochs=10,
    validation_data=(X_test, y_test),
    batch_size=64
)

Epoch 1/10
938/938 [==============================] - 57s 60ms/step - loss: 0.2113 - accuracy: 0.9342 - val_loss: 0.0700 - val_accuracy: 0.9810
Epoch 2/10
938/938 [==============================] - 59s 63ms/step - loss: 0.0578 - accuracy: 0.9837 - val_loss: 0.0518 - val_accuracy: 0.9852
Epoch 3/10
938/938 [==============================] - 57s 61ms/step - loss: 0.0449 - accuracy: 0.9879 - val_loss: 0.0885 - val_accuracy: 0.9794
Epoch 4/10
938/938 [==============================] - 56s 60ms/step - loss: 0.0359 - accuracy: 0.9904 - val_loss: 0.0610 - val_accuracy: 0.9859
Epoch 5/10
938/938 [==============================] - 57s 60ms/step - loss: 0.0307 - accuracy: 0.9916 - val_loss: 0.0885 - val_accuracy: 0.9830
Epoch 6/10
938/938 [==============================] - 66s 71ms/step - loss: 0.0288 - accuracy: 0.9927 - val_loss: 0.0659 - val_accuracy: 0.9856
Epoch 7/10
938/938 [==============================] - 62s 66ms/step - loss: 0.0288 - accuracy: 0.9932 - val_loss: 0.0711 - val_accuracy:

KeyboardInterrupt: 

In [ ]:
custom_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,(3,3), activation='relu', input_shape=(28,28,1)),#we're dealing w colored images so we need 3 channels, we're using 32 filters, 3x3 size (shape), then pick activation function (ReLU), then can use soft_max at the end (standard practice w reLu)
    tf.keras.layers.MaxPooling2D(2,2), #max_pooling layer
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), #don't have to specify input shape for 2nd layer, just first. This one can have bigger filter.
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'), #most models will have 3 to 8 convolution layers
    tf.keras.layers.Flatten(), #should have at least one dense layer in your. model
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax') #this needs to have 10 layers because we have 10 categories. Adding the softmax at the end as mentioned before
])

In [ ]:
custom_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 13, 13, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 3, 3, 128)         73856     
                                                                 
 flatten_2 (Flatten)         (None, 1152)             

In [ ]:
custom_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
custom_model_history = custom_model.fit(
    X_train, y_train,
    epochs=10,
    validation_data=(X_test, y_test),
    batch_size=64)

Epoch 1/10
938/938 [==============================] - 8s 6ms/step - loss: 0.1273 - accuracy: 0.9603 - val_loss: 0.0603 - val_accuracy: 0.9798
Epoch 2/10
938/938 [==============================] - 8s 9ms/step - loss: 0.0330 - accuracy: 0.9897 - val_loss: 0.0370 - val_accuracy: 0.9880
Epoch 3/10
938/938 [==============================] - 6s 6ms/step - loss: 0.0216 - accuracy: 0.9932 - val_loss: 0.0374 - val_accuracy: 0.9881
Epoch 4/10
938/938 [==============================] - 7s 7ms/step - loss: 0.0155 - accuracy: 0.9950 - val_loss: 0.0361 - val_accuracy: 0.9898
Epoch 5/10
938/938 [==============================] - 6s 6ms/step - loss: 0.0131 - accuracy: 0.9958 - val_loss: 0.0291 - val_accuracy: 0.9931
Epoch 6/10
938/938 [==============================] - 6s 6ms/step - loss: 0.0111 - accuracy: 0.9964 - val_loss: 0.0603 - val_accuracy: 0.9875
Epoch 7/10
938/938 [==============================] - 7s 7ms/step - loss: 0.0092 - accuracy: 0.9971 - val_loss: 0.0339 - val_accuracy: 0.9916
Epoch 

If both val and accuracy are high it's okay

In [ ]:
vgg_model = tf.keras.applications.VGG16(weights=None, input_shape=(32,32,1), classes=10)

In [ ]:
inception_model = tf.keras.applications.InceptionV3(weights=None, input_shape=(75,75,1), classes = 10)

In [ ]:
vgg_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 1)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        640       
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [ ]:
inception_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 75, 75, 1)]          0         []                            
                                                                                                  
 conv2d_11 (Conv2D)          (None, 37, 37, 32)           288       ['input_3[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 37, 37, 32)           96        ['conv2d_11[0][0]']           
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 37, 37, 32)           0         ['batch_normalizati

In [ ]:
vgg_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
vgg_history = vgg_model.fit(
    X_train, y_train,
    epochs=10,
    validation_data=(X_test, y_test),
    batch_size = 64
)

Epoch 1/10


ValueError: ignored